<a href="https://colab.research.google.com/github/Lookieman/home_projects/blob/main/ThinkOnwardsComp/Phase2a_Unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This phase trains a U-Net model to get a baseline MAPE.

The model uses the flag USE_ADAPTIVE_MODEL to switch between:

a. Mode that resizes the training data ((10001, 31) → (300, 1259) via interpolation), stack as a 5-channel i/p as baseline.

b. Model that uses "adaptive-inside" model when Flag is True

Objective is to test code and also compare MAPE to be used as baseline

In [ ]:
#Phase2a
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
import torch.nn.functional as F
import numpy as np
import os
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import time
from tqdm import tqdm
import gc
from scipy import ndimage

In [ ]:
# Configuration
USE_ADAPTIVE_MODEL = True  # Set to True for adaptive model
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 16
LEARNING_RATE = 1e-4
NUM_EPOCHS = 30
ENSEMBLE_SIZE = 3
K_FOLDS = 5
DEBUG_ACTIVE=False


print(f"Using device: {DEVICE}")
print(f"Model type: {'Adaptive' if USE_ADAPTIVE_MODEL else 'Resize'}")

Using device: cuda
Model type: Adaptive


In [ ]:
class SeismicDataset(Dataset):
    def __init__(self, data_dir, sample_indices, use_adaptive=False):
        self.data_dir = data_dir
        self.sample_indices = sample_indices
        self.use_adaptive = use_adaptive
        self.receiver_files = [
            'receiver_data_src_1.npy',
            'receiver_data_src_75.npy',
            'receiver_data_src_150.npy',
            'receiver_data_src_225.npy',
            'receiver_data_src_300.npy'
        ]

    def __len__(self):
        return len(self.sample_indices)

    def __getitem__(self, idx):
        sample_idx = self.sample_indices[idx]
        sample_dir = os.path.join(self.data_dir, f'TrainingData_{sample_idx}')

        # Load receiver data (5 files)
        receiver_data = []
        for file_name in self.receiver_files:
            file_path = os.path.join(sample_dir, file_name)
            data = np.load(file_path).astype(np.float32)  # (10001, 31)
            receiver_data.append(data)

        # Load target velocity model
        target_path = os.path.join(sample_dir, 'vp_model.npy')
        target = np.load(target_path).astype(np.float32)  # (300, 1259)

        if self.use_adaptive:
            # Process for adaptive model
            processed_inputs = []
            for data in receiver_data:
                # 1D conv + maxpool simulation using numpy
                # Downsample from 10001 to ~313 (factor of ~32)
                downsampled = data[::32, :]  # (313, 31)

                # Zero pad from 31 to 32 channels
                if downsampled.shape[1] == 31:
                    padded = np.pad(downsampled, ((0, 0), (0, 1)), mode='constant')  # (313, 32)
                else:
                    padded = downsampled

                # Reshape to make it more compact 2D
                # We'll treat this as (313, 32) for now and let the model handle it
                processed_inputs.append(padded.T)  # (32, 313) for easier processing

            # Stack all 5 processed inputs
            input_tensor = np.stack(processed_inputs, axis=0)  # (5, 32, 313)
        else:
            # Process for resize model
            processed_inputs = []
            for data in receiver_data:
                # Resize (10001, 31) to (300, 1259)
                resized = ndimage.zoom(data, (300/10001, 1259/31), order=1)
                processed_inputs.append(resized)

            # Stack all 5 processed inputs
            input_tensor = np.stack(processed_inputs, axis=0)  # (5, 300, 1259)

        return torch.from_numpy(input_tensor), torch.from_numpy(target)


In [ ]:
from math import e
# Attention Block
class AttentionBlock(nn.Module):
    def __init__(self, in_channels):
        super(AttentionBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels // 8, 1)
        self.conv2 = nn.Conv2d(in_channels, in_channels // 8, 1)
        self.conv3 = nn.Conv2d(in_channels, in_channels, 1)
        self.gamma = nn.Parameter(torch.zeros(1))
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        batch_size, channels, height, width = x.size()

        query = self.conv1(x).view(batch_size, -1, width * height).permute(0, 2, 1)
        key = self.conv2(x).view(batch_size, -1, width * height)
        value = self.conv3(x).view(batch_size, -1, width * height)

        attention = torch.bmm(query, key)
        attention = self.softmax(attention)

        out = torch.bmm(value, attention.permute(0, 2, 1))
        out = out.view(batch_size, channels, height, width)

        return self.gamma * out + x

# Double Convolution Block
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

# U-Net Model
class UNet(nn.Module):
    def __init__(self, in_channels=5, out_channels=1, use_adaptive=False):
        super(UNet, self).__init__()
        self.use_adaptive = use_adaptive

        if use_adaptive:
            # First process the irregular input
            self.input_processor = nn.Sequential(
                nn.Conv2d(5, 16, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
                nn.Conv2d(16, 32, kernel_size=3, padding=1),
                nn.ReLU(inplace=True)
            )
            # Input will be (5, 32, 313), output will be (32, 32, 313)
            in_channels = 32

        # Encoder
        self.enc1 = DoubleConv(in_channels, 64)
        self.pool1 = nn.MaxPool2d(2)
        self.enc2 = DoubleConv(64, 128)
        self.pool2 = nn.MaxPool2d(2)
        self.enc3 = DoubleConv(128, 256)
        self.pool3 = nn.MaxPool2d(2)
        self.enc4 = DoubleConv(256, 512)
        self.pool4 = nn.MaxPool2d(2)

        # Bottleneck with attention
        self.bottleneck = DoubleConv(512, 1024)
        self.attention = AttentionBlock(1024)

        # Decoder
        self.up4 = nn.ConvTranspose2d(1024, 512, 2, stride=2)
        self.dec4 = DoubleConv(1024, 512)
        self.up3 = nn.ConvTranspose2d(512, 256, 2, stride=2)
        self.dec3 = DoubleConv(512, 256)
        self.up2 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.dec2 = DoubleConv(256, 128)
        self.up1 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.dec1 = DoubleConv(128, 64)

        # Final output
        self.final_conv = nn.Conv2d(64, out_channels, 1)

        if use_adaptive:
            # Final upsampling to reach (300, 1259)
            self.final_upsample = nn.Sequential(
                nn.ConvTranspose2d(1, 1, kernel_size=4, stride=2, padding=1),
                nn.ReLU(inplace=True)
            )

    def forward(self, x):
        if self.use_adaptive:
            # Process irregular input first
            x = self.input_processor(x)  # (batch, 32, 32, 313)

        # Encoder
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool1(e1))
        e3 = self.enc3(self.pool2(e2))
        e4 = self.enc4(self.pool3(e3))

        # Bottleneck
        b = self.bottleneck(self.pool4(e4))
        b = self.attention(b)

        # Decoder with skip connections
        d4 = self.up4(b)
        d4 = match_tensor_size(d4, e4)
        d4 = torch.cat([d4, e4], dim=1)
        d4 = self.dec4(d4)

        d3 = self.up3(d4)
        d3 = match_tensor_size(d3, e3)
        d3 = torch.cat([d3, e3], dim=1)
        d3 = self.dec3(d3)

        d2 = self.up2(d3)
        d2 = match_tensor_size(d2, e2)
        d2 = torch.cat([d2, e2], dim=1)
        d2 = self.dec2(d2)

        d1 = self.up1(d2)
        d1 = match_tensor_size(d1, e1)
        d1 = torch.cat([d1, e1], dim=1)
        d1 = self.dec1(d1)

        # Final output
        output = self.final_conv(d1)

        if self.use_adaptive:
            # Upsample to final target size (300, 1259)
            output = F.interpolate(output, size=(300, 1259), mode='bilinear', align_corners=False)

        return output

In [ ]:
class KFoldCrossValidator:
    def __init__(self, data_dir, result_dir, k_folds=5):
        self.data_dir = Path(data_dir)
        self.result_dir = Path(result_dir)
        self.k_folds = k_folds
        self.fold_results = []
        self.best_models = []

        # Create results directory
        self.result_dir.mkdir(parents=True, exist_ok=True)

    def create_data_splits(self, dataset_size=2000):
        """Create K-fold splits from training data indices"""
        indices = np.arange(1, dataset_size + 1)  # 1 to 2000 for TrainingData_<i>
        kfold = KFold(n_splits=self.k_folds, shuffle=True, random_state=42)

        splits = []
        for fold, (train_idx, val_idx) in enumerate(kfold.split(indices)):
            # Convert back to sample indices (1-based)
            train_samples = indices[train_idx]
            val_samples = indices[val_idx]

            splits.append({
                'fold': fold,
                'train_indices': train_samples,
                'val_indices': val_samples,
                'train_size': len(train_samples),
                'val_size': len(val_samples)
            })
            print(f"Fold {fold}: Train={len(train_samples)}, Val={len(val_samples)}")

        return splits

    def train_single_fold(self, fold_info, batch_size=8, num_epochs=50, learning_rate=LEARNING_RATE):
        """Train a single fold from scratch using your training function style"""
        fold_num = fold_info['fold']
        print(f"\n=== Training Fold {fold_num} ===")

        # Create fresh model instance
        model = UNet(in_channels=5, out_channels=1, use_adaptive=USE_ADAPTIVE_MODEL)
        model.to(DEVICE)

        # Create dataset and data loaders using your dataset class
        train_dataset = SeismicDataset(self.data_dir, fold_info['train_indices'], use_adaptive=USE_ADAPTIVE_MODEL)
        val_dataset = SeismicDataset(self.data_dir, fold_info['val_indices'], use_adaptive=USE_ADAPTIVE_MODEL)

        train_loader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            shuffle=True,
            num_workers=2,
            pin_memory=True
        )
        val_loader = DataLoader(
            val_dataset,
            batch_size=batch_size,
            shuffle=False,
            num_workers=2,
            pin_memory=True
        )

        # Setup training (matching your style)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5)

        # Training tracking
        train_losses = []
        val_losses = []
        best_val_loss = float('inf')
        best_model_path = self.result_dir / f'best_model_fold_{fold_num}.pth'

        for epoch in range(num_epochs):
            # Training phase (matching your debug style)
            model.train()
            train_loss = 0.0
            train_progress = tqdm(train_loader, desc=f'Fold {fold_num} Epoch {epoch+1}/{num_epochs} [Train]')

            for batch_idx, (inputs, targets) in enumerate(train_progress):
                try:
                    # Debug for first few batches
                    if DEBUG_ACTIVE and batch_idx < 3:
                        print(f"\n*** DEBUG Fold {fold_num} Batch {batch_idx}: Input shape: {inputs.shape}, Target shape: {targets.shape} ***\n")
                        if torch.cuda.is_available():
                            print(f"*** DEBUG: GPU memory before forward: {torch.cuda.memory_allocated()/1e9:.2f}GB ***\n")

                    if DEBUG_ACTIVE and fold_num != 0:
                      # Check if data loading is the issue
                      print(f"\n*** DEBUG: Batch {batch_idx} - Data loaded successfully ***")

                    inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
                    targets = targets.unsqueeze(1)  # Add channel dimension
                    if DEBUG_ACTIVE and fold_num != 0:
                      print(f"*** DEBUG: Batch {batch_idx} - Data moved to GPU ***")

                    optimizer.zero_grad()
                    if DEBUG_ACTIVE and fold_num != 0 :
                       print(f"*** DEBUG: Batch {batch_idx} - Optimizer zeroed ***")

                    outputs = model(inputs)
                    if DEBUG_ACTIVE and fold_num != 0:
                       print(f"*** DEBUG: Batch {batch_idx} - Forward pass complete ***")

                    loss = mape_loss(outputs, targets)
                    if DEBUG_ACTIVE and fold_num != 0:
                       print(f"*** DEBUG: Batch {batch_idx} - Loss calculated: {loss.item():.4f} ***")

                    loss.backward()
                    if DEBUG_ACTIVE and fold_num != 0:
                       print(f"*** DEBUG: Batch {batch_idx} - Backward pass complete ***")

                    optimizer.step()
                    if DEBUG_ACTIVE and fold_num != 0:
                      print(f"*** DEBUG: Batch {batch_idx} - Optimizer step complete ***")

                    train_loss += loss.item()
                    train_progress.set_postfix({'Loss': f'{loss.item():.4f}'})

                    # Debug progress
                    if batch_idx % 10 == 0:
                      torch.cuda.empty_cache()
                      print(f"*** DEBUG: Batch {batch_idx} COMPLETED - Memory cleared ***")

                except Exception as e:
                    print(f"*** ERROR in Fold {fold_num} batch {batch_idx}: {str(e)} ***")
                    raise e

            avg_train_loss = train_loss / len(train_loader)
            train_losses.append(avg_train_loss)

            # Validation phase
            model.eval()
            val_loss = 0.0
            val_batch_count = 0

            with torch.no_grad():
                val_progress = tqdm(val_loader, desc=f'Fold {fold_num} Epoch {epoch+1}/{num_epochs} [Val]')

                for batch_idx, (inputs, targets) in enumerate(val_progress):
                    try:
                        if DEBUG_ACTIVE and fold_num != 0:
                          print(f"\n*** DEBUG: Validation batch {batch_idx} starting ***")

                        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
                        targets = targets.unsqueeze(1)

                        outputs = model(inputs)
                        loss = mape_loss(outputs, targets)
                        val_loss += loss.item()
                        val_batch_count += 1
                        val_progress.set_postfix({'Loss': f'{loss.item():.4f}'})

                        if DEBUG_ACTIVE and fold_num != 0:
                          print(f"*** DEBUG: Validation batch {batch_idx} completed ***")

                        if batch_idx % 5 == 0:
                          torch.cuda.empty_cache()

                    except Exception as e:
                        print(f"*** ERROR in Fold {fold_num} validation batch {batch_idx}: {str(e)} ***")
                        raise e

            avg_val_loss = val_loss / val_batch_count if val_batch_count > 0 else float('inf')
            val_losses.append(avg_val_loss)

            scheduler.step(avg_val_loss)

            print(f'Fold {fold_num} Epoch {epoch+1}/{num_epochs}:')
            print(f'  Train MAPE: {avg_train_loss:.4f}%')
            print(f'  Val MAPE: {avg_val_loss:.4f}%')
            print(f'  LR: {optimizer.param_groups[0]["lr"]:.6f}')
            print('-' * 50)

            # Save best model
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                torch.save({
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'epoch': epoch,
                    'val_loss': best_val_loss,
                    'fold': fold_num
                }, best_model_path)
                print(f'Fold {fold_num}: New best model saved with Val MAPE = {best_val_loss:.4f}%')

            # Early stopping
            if optimizer.param_groups[0]['lr'] < 1e-7:
                print(f'Fold {fold_num}: Early stopping due to low learning rate')
                break

        # Clean up GPU memory
        del model, optimizer, scheduler
        torch.cuda.empty_cache()
        gc.collect()

        return {
            'fold': fold_num,
            'best_val_loss': best_val_loss,
            'train_losses': train_losses,
            'val_losses': val_losses,
            'model_path': best_model_path
        }

    def run_cross_validation(self, batch_size=8, num_epochs=50, learning_rate=1e-4):
        """Run complete K-fold cross-validation"""
        print(f"Starting {self.k_folds}-Fold Cross-Validation")

        # Create data splits
        splits = self.create_data_splits()

        # Train each fold
        for fold_info in splits:
            fold_result = self.train_single_fold(
                fold_info,
                batch_size=batch_size,
                num_epochs=num_epochs,
                learning_rate=learning_rate
            )
            self.fold_results.append(fold_result)

            print(f"Fold {fold_result['fold']} completed with Val MAPE: {fold_result['best_val_loss']:.4f}%")

        # Analyze results
        self.analyze_cv_results()

        return self.fold_results

    def analyze_cv_results(self):
        """Analyze cross-validation results and identify best models"""
        val_scores = [result['best_val_loss'] for result in self.fold_results]

        mean_score = np.mean(val_scores)
        std_score = np.std(val_scores)
        best_fold = np.argmin(val_scores)

        print(f"\n=== Cross-Validation Results ===")
        print(f"Mean Val MAPE: {mean_score:.4f}% ± {std_score:.4f}%")
        print(f"Best Fold: {best_fold} with Val MAPE: {val_scores[best_fold]:.4f}%")
        print(f"Individual Fold Scores: {[f'{score:.4f}%' for score in val_scores]}")

        # Save results
        results_file = self.result_dir / 'cv_results.txt'
        with open(results_file, 'w') as f:
            f.write(f"K-Fold Cross-Validation Results (k={self.k_folds})\n")
            f.write(f"Mean Val MAPE: {mean_score:.4f}% ± {std_score:.4f}%\n")
            f.write(f"Best Fold: {best_fold} with Val MAPE: {val_scores[best_fold]:.4f}%\n")
            for i, score in enumerate(val_scores):
                f.write(f"Fold {i}: {score:.4f}%\n")

        return {
            'mean_score': mean_score,
            'std_score': std_score,
            'best_fold': best_fold,
            'best_score': val_scores[best_fold]
        }

    def create_ensemble_model(self, top_k=ENSEMBLE_SIZE):
        """Create ensemble from top k performing folds"""
        # Sort folds by validation performance
        sorted_folds = sorted(self.fold_results, key=lambda x: x['best_val_loss'])
        top_folds = sorted_folds[:top_k]

        print(f"\nCreating ensemble from top {top_k} folds:")
        for i, fold in enumerate(top_folds):
            print(f"  {i+1}. Fold {fold['fold']}: {fold['best_val_loss']:.4f}%")

        # Load top models
        ensemble_models = []

        for fold in top_folds:
            model = UNet(in_channels=5, out_channels=1, use_adaptive=True)
            checkpoint = torch.load(fold['model_path'], map_location=DEVICE)
            model.load_state_dict(checkpoint['model_state_dict'])
            model.to(DEVICE)
            model.eval()
            ensemble_models.append(model)

        return ensemble_models, [fold['best_val_loss'] for fold in top_folds]

    def create_all_folds_ensemble(self):
        """Create ensemble from ALL folds (for comparison)"""
        print(f"\nCreating ensemble from ALL {len(self.fold_results)} folds:")
        for i, fold in enumerate(self.fold_results):
            print(f"  {i+1}. Fold {fold['fold']}: {fold['best_val_loss']:.4f}%")

        # Load ALL models
        ensemble_models = []

        for fold in self.fold_results:
            model = UNet(in_channels=5, out_channels=1, use_adaptive=True)
            checkpoint = torch.load(fold['model_path'], map_location=DEVICE)
            model.load_state_dict(checkpoint['model_state_dict'])
            model.to(DEVICE)
            model.eval()
            ensemble_models.append(model)

        # Equal weights for simple averaging
        equal_weights = [1.0] * len(self.fold_results)

        return ensemble_models, equal_weights

    def create_weighted_all_folds_ensemble(self):
        """Create ensemble from ALL folds with inverse-loss weighting"""
        print(f"\nCreating WEIGHTED ensemble from ALL {len(self.fold_results)} folds:")

        # Load ALL models
        ensemble_models = []
        fold_losses = []

        for fold in self.fold_results:
            model = UNet(in_channels=5, out_channels=1, use_adaptive=True)
            checkpoint = torch.load(fold['model_path'], map_location=DEVICE)
            model.load_state_dict(checkpoint['model_state_dict'])
            model.to(DEVICE)
            model.eval()
            ensemble_models.append(model)
            fold_losses.append(fold['best_val_loss'])
            print(f"  Fold {fold['fold']}: {fold['best_val_loss']:.4f}%")

        return ensemble_models, fold_losses

In [ ]:
class TestInferencePipeline:
    def __init__(self, test_dir):
        self.test_dir = Path(test_dir)
        self.device = DEVICE
        self.receiver_files = [
            'receiver_data_src_1.npy',
            'receiver_data_src_75.npy',
            'receiver_data_src_150.npy',
            'receiver_data_src_225.npy',
            'receiver_data_src_300.npy'
        ]

    def load_test_sample(self, sample_id):
        """Load a single test sample (5 receiver files) - matching your preprocessing exactly"""
        sample_dir = self.test_dir / f"TestData_{sample_id}"

        # Load the 5 receiver files
        receiver_data = []
        for file_name in self.receiver_files:
            file_path = sample_dir / file_name
            if not file_path.exists():
                raise FileNotFoundError(f"Missing file: {file_path}")

            data = np.load(file_path).astype(np.float32)  # (10001, 31)
            receiver_data.append(data)

        # Apply EXACT same preprocessing as your SeismicDataset adaptive processing
        processed_inputs = []
        for data in receiver_data:
            # 1D conv + maxpool simulation using numpy
            # Downsample from 10001 to ~313 (factor of ~32)
            downsampled = data[::32, :]  # (313, 31)

            # Zero pad from 31 to 32 channels
            if downsampled.shape[1] == 31:
                padded = np.pad(downsampled, ((0, 0), (0, 1)), mode='constant')  # (313, 32)
            else:
                padded = downsampled

            # Reshape to make it more compact 2D
            # We'll treat this as (313, 32) for now and let the model handle it
            processed_inputs.append(padded.T)  # (32, 313) for easier processing

        # Stack all 5 processed inputs
        input_tensor = np.stack(processed_inputs, axis=0)  # (5, 32, 313)

        return torch.from_numpy(input_tensor)

    def predict_single_model(self, model, input_tensor):
        """Generate prediction using a single model"""
        model.eval()
        with torch.no_grad():
            if len(input_tensor.shape) == 3:
                input_tensor = input_tensor.unsqueeze(0)  # Add batch dimension

            input_tensor = input_tensor.to(self.device, non_blocking=True)
            prediction = model(input_tensor)

            # Ensure output is float64 and correct shape
            prediction = prediction.cpu().numpy().astype(np.float64)

            # Remove batch dimension and verify shape
            prediction = prediction.squeeze()
            if prediction.shape != (300, 1259):
                raise ValueError(f"Prediction shape {prediction.shape} doesn't match required (300, 1259)")

            return prediction

    def predict_ensemble(self, models, weights, input_tensor):
        """Generate ensemble prediction with weighted averaging"""
        predictions = []

        for model in models:
            pred = self.predict_single_model(model, input_tensor)
            predictions.append(pred)

        # Handle different weight types
        if all(w == 1.0 for w in weights):
            # Simple averaging (all weights are 1.0)
            ensemble_pred = np.mean(predictions, axis=0)
        else:
            # Weighted average (using inverse loss as weights)
            ensemble_pred = np.zeros_like(predictions[0])
            # Convert MAPE losses to weights (lower loss = higher weight)
            inv_weights = [1.0 / (w + 1e-8) for w in weights]
            total_weight = sum(inv_weights)

            for pred, weight in zip(predictions, inv_weights):
                ensemble_pred += pred * (weight / total_weight)

        return ensemble_pred.astype(np.float64)

    def generate_submission(self, model_or_ensemble, output_path, batch_size=4, weights=None):
        """Generate complete test submission file"""
        print(f"Generating test predictions for TestData_1 to TestData_150...")

        predictions = {}

        # Determine if single model or ensemble
        is_ensemble = isinstance(model_or_ensemble, list)
        if is_ensemble and weights is None:
            weights = [1.0] * len(model_or_ensemble)  # Equal weights if not provided

        # Process samples individually (easier for debugging)
        for sample_id in tqdm(range(1, 151), desc="Processing test samples"):
            try:
                # Load and preprocess single sample
                sample_data = self.load_test_sample(sample_id)

                # Generate prediction
                if is_ensemble:
                    pred = self.predict_ensemble(model_or_ensemble, weights, sample_data)
                else:
                    pred = self.predict_single_model(model_or_ensemble, sample_data)

                # Store with correct naming convention
                predictions[f"TestData_{sample_id}"] = pred

            except Exception as e:
                print(f"Error processing TestData_{sample_id}: {e}")
                continue

        # Save submission file
        output_path = Path(output_path)
        np.savez_compressed(output_path, **predictions)

        print(f"Submission file saved: {output_path}")
        print(f"Generated predictions for {len(predictions)} test samples")

        # Verify submission format
        self.verify_submission(output_path)

        return predictions

    def verify_submission(self, submission_path):
        """Verify submission file format"""
        data = np.load(submission_path)

        print(f"\n=== Submission Verification ===")
        print(f"Total samples: {len(data.files)}")

        # Check a few samples
        for i, key in enumerate(sorted(data.files)[:5]):
            pred = data[key]
            print(f"  {key}: shape={pred.shape}, dtype={pred.dtype}")

            # Verify requirements
            assert pred.shape == (300, 1259), f"Wrong shape for {key}: {pred.shape}"
            assert pred.dtype == np.float64, f"Wrong dtype for {key}: {pred.dtype}"

        print("✓ Submission format verified!")

In [ ]:
# Helper function to match tensor sizes for skip connections
def match_tensor_size(tensor1, tensor2):
    """Match the spatial dimensions of tensor1 to tensor2 by cropping or padding."""
    _, _, h1, w1 = tensor1.shape
    _, _, h2, w2 = tensor2.shape

    # Calculate differences
    dh = h2 - h1
    dw = w2 - w1

    if dh > 0 or dw > 0:
        # Pad tensor1 if it's smaller
        pad_h = max(0, dh)
        pad_w = max(0, dw)
        tensor1 = F.pad(tensor1, (0, pad_w, 0, pad_h))
    elif dh < 0 or dw < 0:
        # Crop tensor1 if it's larger
        tensor1 = tensor1[:, :, :h2, :w2]

    return tensor1

# MAPE Loss Function
def mape_loss(predictions, targets, epsilon=1e-8):
    targets_safe = torch.clamp(torch.abs(targets), min=epsilon)
    return torch.mean(torch.abs((targets - predictions) / targets_safe)) * 100

# Main execution functions
def run_kfold_cv(data_dir, result_dir, k_folds=K_FOLDS):
    """Run K-fold cross-validation"""
    cv = KFoldCrossValidator(
        data_dir=data_dir,
        result_dir=result_dir,
        k_folds=k_folds
    )

    # Run cross-validation with Phase 1 settings
    fold_results = cv.run_cross_validation(
        batch_size=BATCH_SIZE,
        num_epochs=NUM_EPOCHS,
        learning_rate=LEARNING_RATE
    )

    # Create ensemble from top folds
    ensemble_models, ensemble_weights = cv.create_ensemble_model(top_k=ENSEMBLE_SIZE)

    return cv, ensemble_models, ensemble_weights

def generate_test_submission(test_dir, models, weights, output_path):

    """Generate test submission using ensemble"""
    pipeline = TestInferencePipeline(test_dir)

    predictions = pipeline.generate_submission(
        model_or_ensemble=models,
        output_path=output_path,
        weights=weights
    )

    return predictions

In [ ]:
# Main execution ADAPTIVE K-Fold
if __name__ == "__main__":

    # Set data directory
    from google.colab import drive
    drive.mount('/content/drive')
    data_dir = Path('/content/drive/MyDrive/ThinkOnward/Data/Train')  # Adjust path as needed
    result_dir= Path('/content/drive/MyDrive/ThinkOnward/Result/Phase2a')
    test_dir = Path('/content/drive/MyDrive/ThinkOnward/Data/Test')

    print("=== Phase 2a: K-Fold CV + Test Inference ===")

    # Step 1: Run K-Fold Cross-Validation
    print("Step 1: Running K-Fold Cross-Validation...")
    cv, ensemble_models, ensemble_weights = run_kfold_cv(
        data_dir=data_dir,
        result_dir=result_dir,
        k_folds=K_FOLDS
    )

     # Step 2: Create and test ALL ensemble approaches
    print("\n" + "="*60)
    print("Step 2: Comparing Different Ensemble Strategies")
    print("="*60)

    # Ensemble 1: Top 3 folds (current approach)
    print("\n🏆 Ensemble 1: Top 3 Folds (Weighted)")
    top3_models, top3_weights = cv.create_ensemble_model(top_k=3)

    # Ensemble 2: All 5 folds (simple averaging)
    print("\n📊 Ensemble 2: All 5 Folds (Simple Average)")
    all5_models, all5_weights = cv.create_all_folds_ensemble()

    # Ensemble 3: All 5 folds (weighted averaging)
    print("\n⚖️ Ensemble 3: All 5 Folds (Weighted Average)")
    all5_weighted_models, all5_weighted_weights = cv.create_weighted_all_folds_ensemble()

    # Step 3: Generate submissions for all approaches
    print("\n" + "="*60)
    print("Step 3: Generating Test Submissions")
    print("="*60)

    # Submission 1: Top 3 weighted
    print("\n🔄 Generating Top 3 Ensemble submission...")
    submission1_path = result_dir / 'submission_top3_weighted.npz'
    predictions1 = generate_test_submission(
        test_dir=test_dir,
        models=top3_models,
        weights=top3_weights,
        output_path=submission1_path
    )

    # Submission 2: All 5 simple average
    print("\n🔄 Generating All 5 Simple Average submission...")
    submission2_path = result_dir / 'submission_all5_simple.npz'
    predictions2 = generate_test_submission(
        test_dir=test_dir,
        models=all5_models,
        weights=all5_weights,  # Equal weights
        output_path=submission2_path
    )

    # Submission 3: All 5 weighted average
    print("\n🔄 Generating All 5 Weighted Average submission...")
    submission3_path = result_dir / 'submission_all5_weighted.npz'
    predictions3 = generate_test_submission(
        test_dir=test_dir,
        models=all5_weighted_models,
        weights=all5_weighted_weights,
        output_path=submission3_path
    )

    # Step 4: Best single model (for comparison)
    print("\n🏅 Generating Best Single Model submission...")
    best_fold_idx = min(range(len(cv.fold_results)), key=lambda i: cv.fold_results[i]['best_val_loss'])
    best_model_path = cv.fold_results[best_fold_idx]['model_path']

    best_model = UNet(in_channels=5, out_channels=1, use_adaptive=True)
    checkpoint = torch.load(best_model_path, map_location=DEVICE)
    best_model.load_state_dict(checkpoint['model_state_dict'])
    best_model.to(DEVICE)
    best_model.eval()

    single_submission_path = result_dir / 'submission_single_best.npz'
    single_predictions = generate_test_submission(
        test_dir=test_dir,
        models=best_model,
        weights=None,
        output_path=single_submission_path
    )

    # Step 5: Summary
    print("\n" + "="*60)
    print("🎯 ENSEMBLE COMPARISON SUMMARY")
    print("="*60)
    print(f"📁 Results saved to: {result_dir}")
    print("\n📊 Cross-Validation Results:")
    cv.analyze_cv_results()
    print(f"\n📋 Generated Submissions:")
    print(f"   1. Top 3 Weighted:      {submission1_path}")
    print(f"   2. All 5 Simple Avg:    {submission2_path}")
    print(f"   3. All 5 Weighted Avg:  {submission3_path}")
    print(f"   4. Best Single Model:   {single_submission_path}")

    print(f"\n🧪 TEST ON LEADERBOARD:")
    print(f"   • Upload all 4 submissions to see which ensemble strategy works best")
    print(f"   • Expected ranking: Single > Top3 > All5_Weighted > All5_Simple")
    print(f"   • If All5 beats Top3, then model diversity > individual quality")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
=== Phase 2a: K-Fold CV + Test Inference ===
Step 1: Running K-Fold Cross-Validation...
Starting 5-Fold Cross-Validation
Fold 0: Train=1600, Val=400
Fold 1: Train=1600, Val=400
Fold 2: Train=1600, Val=400
Fold 3: Train=1600, Val=400
Fold 4: Train=1600, Val=400

=== Training Fold 0 ===


Fold 0 Epoch 1/30 [Train]:   1%|          | 1/100 [00:00<01:12,  1.37it/s, Loss=80.9175]

*** DEBUG: Batch 0 COMPLETED - Memory cleared ***


Fold 0 Epoch 1/30 [Train]:  11%|█         | 11/100 [00:03<00:25,  3.53it/s, Loss=66.9912]

*** DEBUG: Batch 10 COMPLETED - Memory cleared ***


...

Submission file saved: /content/drive/MyDrive/ThinkOnward/Result/Phase2a/submission_all5_simple.npz
Generated predictions for 150 test samples

=== Submission Verification ===
Total samples: 150
  TestData_1: shape=(300, 1259), dtype=float64
  TestData_10: shape=(300, 1259), dtype=float64
  TestData_100: shape=(300, 1259), dtype=float64
  TestData_101: shape=(300, 1259), dtype=float64
  TestData_102: shape=(300, 1259), dtype=float64
✓ Submission format verified!

🔄 Generating All 5 Weighted Average submission...
Generating test predictions for TestData_1 to TestData_150...


Processing test samples: 100%|██████████| 150/150 [00:08<00:00, 17.64it/s]


Submission file saved: /content/drive/MyDrive/ThinkOnward/Result/Phase2a/submission_all5_weighted.npz
Generated predictions for 150 test samples

=== Submission Verification ===
Total samples: 150
  TestData_1: shape=(300, 1259), dtype=float64
  TestData_10: shape=(300, 1259), dtype=float64
  TestData_100: shape=(300, 1259), dtype=float64
  TestData_101: shape=(300, 1259), dtype=float64
  TestData_102: shape=(300, 1259), dtype=float64
✓ Submission format verified!

🏅 Generating Best Single Model submission...
Generating test predictions for TestData_1 to TestData_150...


Processing test samples: 100%|██████████| 150/150 [00:03<00:00, 42.30it/s]


Submission file saved: /content/drive/MyDrive/ThinkOnward/Result/Phase2a/submission_single_best.npz
Generated predictions for 150 test samples

=== Submission Verification ===
Total samples: 150
  TestData_1: shape=(300, 1259), dtype=float64
  TestData_10: shape=(300, 1259), dtype=float64
  TestData_100: shape=(300, 1259), dtype=float64
  TestData_101: shape=(300, 1259), dtype=float64
  TestData_102: shape=(300, 1259), dtype=float64
✓ Submission format verified!

🎯 ENSEMBLE COMPARISON SUMMARY
📁 Results saved to: /content/drive/MyDrive/ThinkOnward/Result/Phase2a

📊 Cross-Validation Results:

=== Cross-Validation Results ===
Mean Val MAPE: 4.2482% ± 1.3076%
Best Fold: 3 with Val MAPE: 3.3345%
Individual Fold Scores: ['6.8394%', '3.5438%', '3.6392%', '3.3345%', '3.8842%']

📋 Generated Submissions:
   1. Top 3 Weighted:      /content/drive/MyDrive/ThinkOnward/Result/Phase2a/submission_top3_weighted.npz
   2. All 5 Simple Avg:    /content/drive/MyDrive/ThinkOnward/Result/Phase2a/submission_